# W_GL_ACCOUNT_D ETL Process
### Staging to Dimension - SCD Type 1 with Change Detection

In [ ]:
%sql
-- 1) PARAMETERS: Derive ETL control parameters
CREATE OR REPLACE TEMP VIEW etl_params AS
SELECT
  CASE WHEN COUNT(*) > 0 THEN 'Y' ELSE 'N' END AS IS_INCREMENTAL,
  COALESCE(date_format(MIN(LAST_MAX_DATE), 'yyyy-MM-dd HH:mm:ss'), '1970-01-01 00:00:00') AS LAST_EXTRACT_DATE,
  380 AS DATASOURCE_NUM_ID, '__NOT_APPLICABLE__' AS ETL_USAGE_CODE, '1' AS TENANT_ID,
  2624850 AS ETL_PROC_WID, 12345 AS EXECUTION_ID, 30 AS PRUNE_DAYS
FROM workspace.oracle_edw.w_etl_load_dates
WHERE PACKAGE_NAME = 'SILOS_SIL_GLACCOUNTDIMENSION'
  AND DATASOURCE_NUM_ID = 380 AND ETL_USAGE_CODE = '__NOT_APPLICABLE__' AND COMMITTED = '1';

In [ ]:
%sql
-- 2) STAGING: Read from staging table W_GL_ACCOUNT_DS with segment lookups
CREATE OR REPLACE TEMP VIEW stg_account_with_segments AS
SELECT 
  ds.gl_account_id, ds.account_type, ds.chart_of_accounts_id,
  -- Segment lookups would go here for each segment dimension
  ds.segment1, ds.segment2, ds.segment3, ds.segment4, ds.segment5,
  ds.segment6, ds.segment7, ds.segment8, ds.segment9, ds.segment10,
  ds.segment11, ds.segment12, ds.segment13, ds.segment14, ds.segment15,
  ds.segment16, ds.segment17, ds.segment18, ds.segment19, ds.segment20,
  ds.segment21, ds.segment22, ds.segment23, ds.segment24, ds.segment25,
  ds.segment26, ds.segment27, ds.segment28, ds.segment29, ds.segment30,
  ds.creation_date AS created_on_dt, ds.changed_on_dt,
  ds.changed_on_dt AS aux1_changed_on_dt,
  CAST(NULL AS TIMESTAMP) AS aux2_changed_on_dt,
  CAST(NULL AS TIMESTAMP) AS aux3_changed_on_dt,
  CAST(NULL AS TIMESTAMP) AS aux4_changed_on_dt,
  CASE WHEN ds.delete_flg = 'Y' THEN 'Y' ELSE 'N' END AS delete_flg,
  ds.datasource_num_id, ds.integration_id, ds.tenant_id, ds.x_custom
FROM workspace.oracle_edw.w_gl_account_ds ds;

In [ ]:
%sql
-- 3) CHANGE DETECTION: Identify new and changed records
CREATE OR REPLACE TEMP VIEW stg_with_change_ind AS
SELECT s.*,
  CASE
    WHEN t.row_wid IS NOT NULL
      AND (t.changed_on_dt = s.changed_on_dt OR (t.changed_on_dt IS NULL AND s.changed_on_dt IS NULL))
      AND (t.aux1_changed_on_dt = s.aux1_changed_on_dt OR (t.aux1_changed_on_dt IS NULL AND s.aux1_changed_on_dt IS NULL))
      AND (t.aux2_changed_on_dt = s.aux2_changed_on_dt OR (t.aux2_changed_on_dt IS NULL AND s.aux2_changed_on_dt IS NULL))
      AND (t.aux3_changed_on_dt = s.aux3_changed_on_dt OR (t.aux3_changed_on_dt IS NULL AND s.aux3_changed_on_dt IS NULL))
      AND (t.aux4_changed_on_dt = s.aux4_changed_on_dt OR (t.aux4_changed_on_dt IS NULL AND s.aux4_changed_on_dt IS NULL))
    THEN 'N'  -- No change
    WHEN t.row_wid IS NOT NULL THEN 'U'  -- Update
    ELSE 'I'  -- Insert
  END AS ind_update
FROM stg_account_with_segments s
LEFT JOIN workspace.oracle_edw.w_gl_account_d t
  ON s.datasource_num_id = t.datasource_num_id
  AND s.integration_id = t.integration_id;

In [ ]:
%sql
-- 4) MERGE: Update existing and insert new records
MERGE INTO workspace.oracle_edw.w_gl_account_d AS tgt
USING (
  SELECT * FROM stg_with_change_ind WHERE ind_update IN ('I', 'U')
) AS src
ON tgt.datasource_num_id = src.datasource_num_id
  AND tgt.integration_id = src.integration_id
WHEN MATCHED AND src.ind_update = 'U' THEN UPDATE SET
  tgt.gl_account_id = src.gl_account_id,
  tgt.account_type = src.account_type,
  tgt.chart_of_accounts_id = src.chart_of_accounts_id,
  tgt.segment1 = src.segment1, tgt.segment2 = src.segment2, tgt.segment3 = src.segment3,
  tgt.segment4 = src.segment4, tgt.segment5 = src.segment5, tgt.segment6 = src.segment6,
  tgt.segment7 = src.segment7, tgt.segment8 = src.segment8, tgt.segment9 = src.segment9,
  tgt.segment10 = src.segment10, tgt.segment11 = src.segment11, tgt.segment12 = src.segment12,
  tgt.segment13 = src.segment13, tgt.segment14 = src.segment14, tgt.segment15 = src.segment15,
  tgt.segment16 = src.segment16, tgt.segment17 = src.segment17, tgt.segment18 = src.segment18,
  tgt.segment19 = src.segment19, tgt.segment20 = src.segment20, tgt.segment21 = src.segment21,
  tgt.segment22 = src.segment22, tgt.segment23 = src.segment23, tgt.segment24 = src.segment24,
  tgt.segment25 = src.segment25, tgt.segment26 = src.segment26, tgt.segment27 = src.segment27,
  tgt.segment28 = src.segment28, tgt.segment29 = src.segment29, tgt.segment30 = src.segment30,
  tgt.created_on_dt = src.created_on_dt,
  tgt.changed_on_dt = src.changed_on_dt,
  tgt.aux1_changed_on_dt = src.aux1_changed_on_dt,
  tgt.aux2_changed_on_dt = src.aux2_changed_on_dt,
  tgt.aux3_changed_on_dt = src.aux3_changed_on_dt,
  tgt.aux4_changed_on_dt = src.aux4_changed_on_dt,
  tgt.delete_flg = src.delete_flg,
  tgt.tenant_id = src.tenant_id,
  tgt.x_custom = src.x_custom,
  tgt.w_update_dt = current_timestamp(),
  tgt.etl_proc_wid = (SELECT ETL_PROC_WID FROM etl_params)
WHEN NOT MATCHED AND src.ind_update = 'I' THEN INSERT (
  gl_account_id, account_type, chart_of_accounts_id,
  segment1, segment2, segment3, segment4, segment5, segment6, segment7, segment8, segment9, segment10,
  segment11, segment12, segment13, segment14, segment15, segment16, segment17, segment18, segment19, segment20,
  segment21, segment22, segment23, segment24, segment25, segment26, segment27, segment28, segment29, segment30,
  created_on_dt, changed_on_dt, aux1_changed_on_dt, aux2_changed_on_dt, aux3_changed_on_dt, aux4_changed_on_dt,
  delete_flg, datasource_num_id, integration_id, tenant_id, x_custom,
  w_insert_dt, w_update_dt, etl_proc_wid
) VALUES (
  src.gl_account_id, src.account_type, src.chart_of_accounts_id,
  src.segment1, src.segment2, src.segment3, src.segment4, src.segment5, src.segment6, src.segment7, src.segment8, src.segment9, src.segment10,
  src.segment11, src.segment12, src.segment13, src.segment14, src.segment15, src.segment16, src.segment17, src.segment18, src.segment19, src.segment20,
  src.segment21, src.segment22, src.segment23, src.segment24, src.segment25, src.segment26, src.segment27, src.segment28, src.segment29, src.segment30,
  src.created_on_dt, src.changed_on_dt, src.aux1_changed_on_dt, src.aux2_changed_on_dt, src.aux3_changed_on_dt, src.aux4_changed_on_dt,
  src.delete_flg, src.datasource_num_id, src.integration_id, src.tenant_id, src.x_custom,
  current_timestamp(), current_timestamp(), (SELECT ETL_PROC_WID FROM etl_params)
);

In [ ]:
%sql
-- 5) UPDATE CONTROL TABLE
MERGE INTO workspace.oracle_edw.w_etl_load_dates AS tgt
USING (
  SELECT p.DATASOURCE_NUM_ID, 'SILOS_SIL_GLACCOUNTDIMENSION' AS package_name,
    'W_GL_ACCOUNT_D' AS target_table_name, p.ETL_USAGE_CODE, p.ETL_PROC_WID, p.EXECUTION_ID AS load_plan_id,
    current_timestamp() AS etl_load_date, CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END AS committed,
    date_add(current_timestamp(), -p.PRUNE_DAYS) AS wip_load_start_date FROM etl_params p
) src ON tgt.datasource_num_id = src.datasource_num_id AND tgt.package_name = src.package_name
WHEN MATCHED THEN UPDATE SET tgt.target_table_name = src.target_table_name, tgt.etl_load_date = src.etl_load_date, tgt.committed = src.committed
WHEN NOT MATCHED THEN INSERT VALUES (src.datasource_num_id, src.package_name, src.target_table_name, src.etl_usage_code,
  src.etl_proc_wid, src.load_plan_id, src.wip_load_start_date, NULL, src.etl_load_date, src.committed);

In [ ]:
%sql
-- 6) INSERT HISTORY LOG
INSERT INTO workspace.oracle_edw.w_etl_load_dates_log
SELECT p.DATASOURCE_NUM_ID, 'SILOS_SIL_GLACCOUNTDIMENSION', 'W_GL_ACCOUNT_D', p.ETL_USAGE_CODE,
  p.ETL_PROC_WID, p.EXECUTION_ID, p.ETL_PROC_WID, date_add(current_timestamp(), -p.PRUNE_DAYS), NULL,
  current_timestamp(), CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END FROM etl_params p;

In [ ]:
%sql
-- 7) CLEANUP
DROP VIEW IF EXISTS stg_with_change_ind;
DROP VIEW IF EXISTS stg_account_with_segments;
DROP VIEW IF EXISTS etl_params;